In [1]:
! pip install cohere altair umap-learn -q
! pip install langchain
! pip install python-dotenv
! pip install qdrant-client
! pip install tfds_nightly
! pip install --upgrade --quiet  wikipedia
! pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.3/206.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 14.1 MB/s eta 

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 4.0 MB/s eta 0:00:00


In [1]:
cohere_api_key='zpeSL6UKGjJPNBOdoro7IYIWMhIzDU3OCLXQSlBG'
pine_api_key='0a8e891f-6e65-4c73-9127-45249cb108fa'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import urllib.request
from bs4 import BeautifulSoup

urllib.request.urlretrieve("https://www.geeksforgeeks.org/grep-command-in-unixlinux/?ref=leftbar-rightbar",
                           "/content/drive/My Drive/text_file.txt")

file = open("/content/drive/My Drive/text_file.txt", "r")
contents = file.read()
soup = BeautifulSoup(contents, 'html.parser')

f = open("/content/drive/My Drive/test1.txt", "w")

# traverse paragraphs from soup
for data in soup.find_all("p"):
    sum = data.get_text()
    f.writelines(sum)

f.close()

In [4]:
from langchain.retrievers import WikipediaRetriever
wikipedia_retriever = WikipediaRetriever()
wiki_docs = wikipedia_retriever.get_relevant_documents("Netflix")
text_doc = wiki_docs[0].page_content
text_3_paras = text_doc.split("\n\n")[:3]
text_3_paras = "".join(text_3_paras)
print(text_3_paras)

Netflix is an American subscription video on-demand over-the-top streaming service. The service primarily distributes original and acquired films and television shows from various genres, and it is available internationally in multiple languages.Launched on January 16, 2007, nearly a decade after Netflix, Inc. began its pioneering DVD‑by‑mail movie rental service, Netflix is the most-subscribed video on demand streaming media service, with 260.28 million paid memberships in more than 190 countries as of January 2024. By 2022, "Netflix Original" productions accounted for half of its library in the United States and the namesake company had ventured into other categories, such as video game publishing of mobile games via its flagship service. As of October 2023, Netflix is the 24th most-visited website in the world with 23.66% of its traffic coming from the United States, followed by the United Kingdom at 5.84% and Brazil at 5.64%.
== History ==
=== Launch as a mail-based rental business

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0,length_function = len, )

In [6]:
text_3_para_list = splitter.split_text(text_3_paras)

In [7]:
queries = ["What does Netflix do?",
           "How did Netflix get established"]

In [9]:
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Qdrant
embeddings = CohereEmbeddings(model="multilingual-22-12", cohere_api_key=cohere_api_key)
db = Qdrant.from_texts(text_3_para_list, embeddings, location=":memory:",colleection_name="my_documents",distance_func="Dot")

In [10]:
answers = []
for query in queries:
  docs= db.similarity_search(query)
  answers.append(docs[0].page_content)

In [11]:
answers

['Netflix is an American subscription video on-demand over-the-top streaming service. The service',
 'Netflix was founded by Marc Randolph and Reed Hastings on August 29, 1997, in Scotts Valley,']

In [12]:
pine_api_key='0a8e891f-6e65-4c73-9127-45249cb108fa'